In [1]:
import src.utils.mesh_tools as mt
from src.learning_based.SDF_CNN import CNN_3d_multiple_split
from src.learning_based.CNN_to_PoNQ_or_lite import CNN_to_PoNQ
from src.learning_based.ABC_dataset import make_mask_close

import torch
from meshplot import plot
import igl

In [40]:
device = 'cpu'
state_dict = './data/pretrained_PoNQ_ABC.pt'
example_mesh = './data/bunny.obj'

model = CNN_3d_multiple_split(device=device)

In [41]:
model.load_state_dict(torch.load(state_dict, map_location=device))

<All keys matched successfully>

In [42]:
v, f = igl.read_triangle_mesh(example_mesh)
v = 2*mt.NDCnormalize(v)
grid_n = 33

points = mt.mesh_grid(grid_n, True)
sdf = igl.signed_distance(points, v, f)[0].reshape(grid_n, grid_n, grid_n)

tensor_sdf = torch.tensor(sdf, dtype=torch.float32)[None, None, ...]
tensor_mask = torch.tensor(make_mask_close(sdf, grid_n), dtype=torch.bool).flatten()[None, ...]

In [43]:
ponq = CNN_to_PoNQ(model, tensor_sdf, 33, tensor_mask, device=device, subd=1)

In [44]:
plot(*ponq.min_cut_surface(grid_n/2**2))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0006789…

In [45]:
plot(*mt.mesh_from_voxels(sdf))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0004573…